# Query to get intergenic regions of a TARA sample in the OcéanIA Platform

### 1. Install oceania-query-fasta package

In [ ]:
%pip install oceania-query-fasta
%pip install openpyxl # required to load supplementary info into pandas

### 2. Load supplementary information for samples and data of Ocean Microbial Reference Gene Catalog v2

In [ ]:
import pandas

SUPP_INFO_LINK = "https://zenodo.org/record/3539258/files/Salazar_et_al_2019_Suppl_Info.xlsx?download=1"

table_W1 = pandas.read_excel(SUPP_INFO_LINK, sheet_name="Table_W1")

# Select first sample_id at the surface water layer
selected_sample = table_W1[table_W1.Layer=="SRF"].head(1)
print(selected_sample)

sample_id = selected_sample["PANGAEA sample id"].item()

### 3. Get metadata for the first 10 intergenic regions of size greater than 100 for the selected sample

In [ ]:
from oceania import list_intergenic_regions

intergenic_regions_metadata = list_intergenic_regions(sample_id, min_length=100, page=1, page_size=10)

print(intergenic_regions_metadata)

### 4. Prepare request params

In [ ]:
request_regions = intergenic_regions_metadata[['sequence', 'start', 'stop']].copy()
request_params = []
for row in request_regions.itertuples():
    request_params.append(
        (row[1], int(row[2]), int(row[3]))
    )

print(request_params)

### 5. Perform call to the OcéanIA services

In [ ]:
from oceania import get_sequences_from_fasta

request_result = get_sequences_from_fasta(
    sample_id,
    request_params
)

# request_result is loaded as a pandas.DataFrame
print(request_result)